In [1]:
import numpy as np

In [3]:
# import the base HDC class
from pyhdc import HDC

OSError: libchdc.so: cannot open shared object file: No such file or directory

In [3]:
# import fortran function wrapper
from hdc_fortran_module import test_cpos

First, we'll construct the input "equilibrium" container.

In [4]:
equilibrium = HDC()
equilibrium['profiles_1d/psi'] = np.linspace(0, 1, 5)
equilibrium['time'] = np.array(2.34)

Display the equilibrium using the dump method.

In [5]:
print(equilibrium.dumps())

{
	"profiles_1d" : 
	{
		"psi" : "[0,0.25,0.5,0.75,1]"
	},
	"time" : "2.34"
}


Print nodes using .as_array()

In [6]:
print("equilibrium['time'] -> {}".format(equilibrium['time'].as_array()))
print("equilibrium['profiles_1d/psi'] -> {}".format(equilibrium['profiles_1d/psi'].as_array()))

equilibrium['time'] -> 2.34
equilibrium['profiles_1d/psi'] -> [ 0.    0.25  0.5   0.75  1.  ]


Call a Fortran function that inputs and outputs a container

In [7]:
tree_out = test_cpos(equilibrium)

In [8]:
print(tree_out.dumps())

{
	"distsourceout" : 
	{
		"codeparam" : 
		{
			"codename" : "\"equ2dist\"",
			"codeversion" : "\"1.0\"",
			"output_diag" : "\"Made in Python\"",
			"output_flag" : "0",
			"parameters" : "\"my_code_specific_parameters\""
		},
		"profiles_1d" : 
		{
			"psi" : "[0,0.25,0.5,0.75,1]"
		},
		"source" : 
		{
			"profiles_1d" : 
			{
				"psi" : "[-0,-0.75,-1.5,-2.25,-3]"
			}
		},
		"time" : "2.34"
	}
}


In [9]:
tree_out.json_dump("test.json")

In [10]:
%cat test.json

{
	"distsourceout" : 
	{
		"codeparam" : 
		{
			"codename" : "equ2dist",
			"codeversion" : "1.0",
			"output_diag" : "Made in Python",
			"output_flag" : "0",
			"parameters" : "my_code_specific_parameters"
		},
		"profiles_1d" : 
		{
			"psi" : [0,0.25,0.5,0.75,1]
		},
		"source" : 
		{
			"profiles_1d" : 
			{
				"psi" : [-0,-0.75,-1.5,-2.25,-3]
			}
		},
		"time" : "2.34"
	}
}

Check the effect of the fortran function

In [11]:
assert np.all(tree_out['distsourceout/source/profiles_1d/psi'].as_array() == 
              -3 * equilibrium['profiles_1d/psi'].as_array())